## setup the valid and sample dirs

In [3]:
import os

In [5]:
os.getcwd()

'/home/ubuntu/dogscats'

In [8]:
current_dir = os.getcwd()
data_dir = current_dir + '/data'

In [13]:
%cd $data_dir

/home/ubuntu/dogscats/data


In [15]:
%mkdir valid
%mkdir sample

mkdir: cannot create directory ‘valid’: File exists


In [27]:
from glob import glob
from numpy.random import permutation

In [28]:
%cd $data_dir/train

/home/ubuntu/dogscats/data/train


In [56]:
f = glob('*.jpg')
shuffle = permutation(f)
for i in range(2000): os.rename(shuffle[i], data_dir + '/valid/' + shuffle[i])

In [59]:
%cd $data_dir/sample
%mkdir train
%mkdir test
%mkdir valid
%mkdir result

/home/ubuntu/dogscats/data/sample


In [60]:
from shutil import copyfile

In [61]:
%cd $data_dir/train

/home/ubuntu/dogscats/data/train


In [66]:
f = glob('*.jpg')
shuffle = permutation(f)
for i in range(200): copyfile(shuffle[i], data_dir + '/sample/' + shuffle[i])

## go to the sample dir and set up data subdir with the data copied from train

In [71]:
%cd $data_dir/sample

/home/ubuntu/dogscats/data/sample


In [77]:
f = glob('*.jpg')
shuffle = permutation(f)
for i in range(20): os.rename(shuffle[i], data_dir + '/sample/valid/' + shuffle[i])

In [82]:
f = glob('*.jpg')
shuffle = permutation(f)
for i in range(len(shuffle)): os.rename(shuffle[i], data_dir + '/sample/train/' + shuffle[i])

In [85]:
%cd $data_dir/sample/train
%ls -1 | wc -l
%mkdir dogs
%mkdir cats
%mv cat*.jpg cats/
%mv dog*.jpg dogs/

/home/ubuntu/dogscats/data/sample/train
180


In [87]:
%cd $data_dir/sample/valid
%ls -1 | wc -l
%mkdir dogs
%mkdir cats
%mv cat*.jpg cats/
%mv dog*.jpg dogs/

/home/ubuntu/dogscats/data/sample/valid
20


## setup subdirs in main train and valid dirs

In [90]:
%cd $data_dir/train
%ls -1 | wc -l
%mkdir dogs
%mkdir cats
%mv dog*.jpg dogs/
%mv cat*.jpg cats/

/home/ubuntu/dogscats/data/train
23000


In [92]:
%cd $data_dir/valid
%ls -1 | wc -l
%mkdir dogs
%mkdir cats
%mv dog*.jpg dogs/
%mv cat*.jpg cats/

/home/ubuntu/dogscats/data/valid
2000


In [96]:
%cd $data_dir/test
%ls -1 | wc -l
%mkdir unknown
%mv *.jpg unknown

/home/ubuntu/dogscats/data/test
12501
mkdir: cannot create directory ‘unknown’: File exists


## move some test data from test dir to sample test dir

In [99]:
%cd $data_dir/sample/test
%mkdir unknown

/home/ubuntu/dogscats/data/sample/test


In [104]:
%cd $data_dir/test/unknown
f = glob('*.jpg')
shuffle = permutation(f)
len(shuffle)
for i in range(100): copyfile(shuffle[i], data_dir + '/sample/test/unknown/' + shuffle[i])

/home/ubuntu/dogscats/data/test/unknown


## create and run the model

In [134]:
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Lambda, Flatten, Dense
from keras.optimizers import Adam
import numpy as np

In [130]:
path = data_dir + '/sample/'
batch_size = 4
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(path + '/train/', batch_size=batch_size)
val_batches = gen.flow_from_directory(path + '/valid/', batch_size=batch_size)
imgs, labels = next(batches)

Found 180 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [135]:
mean_px = imgs.mean().astype(np.float32)
std_px = imgs.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [136]:
def get_lin_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(3,256,256)),
        Flatten(),
        Dense(2, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [137]:
lm = get_lin_model()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 256, 256)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [142]:
lm.fit_generator(batches, batches.nb_sample, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
180/180 [==============================] - 1297s - loss: 7.7655 - acc: 0.5111 - val_loss: 5.0759 - val_acc: 0.6500

In [143]:
weights = lm.get_weights()

In [147]:
test_batches = gen.flow_from_directory(path + '/test/')

Found 100 images belonging to 1 classes.


In [148]:
test_imgs, unknown = next(test_batches)

In [150]:
preds = lm.predict(test_imgs,verbose=1)

32/32 [==============================] - 58s


In [151]:
preds

array([[  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.73009325e-16],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.40129846e-45],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   2.52200036e-38],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  4.58865527e-36,   1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  

In [156]:
filenames = test_batches.filenames

In [168]:
from PIL import Image
Image.open(path + '/test/' + filenames[])
#removing filename to keep nb size low

SyntaxError: invalid syntax (<ipython-input-168-643402f0a5b8>, line 2)

In [162]:
lm.save_weights(path + '/result/' + 'ft1.h5')

In [166]:
weights[0].shape

(196608, 2)